## Imports and Environment Variables

In [3]:
import os
from dotenv import load_dotenv

In [4]:
# Load environment variables
load_dotenv()

SUBSCRIPTION_ID = os.getenv('SUBSCRIPTION_ID')

## Feature Engineering

In [5]:
import pandas as pd

# Read the data
df = pd.read_csv('src/data/train.csv', index_col=0)

In [6]:
# Explore the data
print(df.dtypes)
df.sample(10)


Gender                   object
Age                       int64
Driving_License           int64
Region_Code             float64
Previously_Insured        int64
Vehicle_Age              object
Vehicle_Damage           object
Annual_Premium          float64
Policy_Sales_Channel    float64
Vintage                   int64
Response                  int64
dtype: object


,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
id,,,,,,,,,,,
3550454,Female,43,1,15.0,0,> 2 Years,Yes,26892.0,14.0,105,0
5739660,Male,45,1,8.0,0,1-2 Year,Yes,45691.0,26.0,77,1
8402074,Male,27,1,23.0,0,1-2 Year,No,2630.0,157.0,169,0
5105398,Female,22,1,46.0,1,< 1 Year,No,25706.0,152.0,171,0
3688547,Male,53,1,28.0,0,> 2 Years,Yes,45331.0,122.0,233,0
8231014,Male,23,1,15.0,1,< 1 Year,No,30082.0,152.0,121,0
4527275,Female,25,1,15.0,1,< 1 Year,No,37682.0,152.0,184,0
3953899,Female,20,1,35.0,1,< 1 Year,No,43858.0,160.0,216,0
1994285,Male,46,1,47.0,0,1-2 Year,Yes,32756.0,124.0,227,0


## Generate training scripts

In [7]:
# create a folder for the script files
script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

src folder created


In [23]:
%%writefile $script_folder/train.py
# Import libraries
import argparse
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

def parse_args():
    
    # Parse arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--train-data", dest="train_data", type=str, required=True)
    parser.add_argument("--test-data", dest="test_data", type=str, required=True)
    args = parser.parse_args()
    
    return args

def main(args):
    print("Reading data...")
    train_df = read_data(args.train_data)
    test_df = read_data(args.test_data)
    
    categorical_columns = ["Gender", "Vehicle_Age", "Vehicle_Damage"]
    numerical_columns = ["Age", "Annual_Premium"]
    
    print("Preprocessing data and applying encoding to categorical columns...")

    # Apply ordinal encoding to categorical columns
    for col in categorical_columns:
        encoder = OrdinalEncoder()
        train_df[col] = encoder.fit_transform(train_df[col].values.reshape(-1,1))
        test_df[col] = encoder.transform(test_df[col].values.reshape(-1,1))
    
    # Normalize the data
    for col in numerical_columns:
        scaler = MinMaxScaler()
        train_df[col] = scaler.fit_transform(train_df[col].values.reshape(-1,1))
        test_df[col] = scaler.transform(test_df[col].values.reshape(-1,1))
    
    X_train = train_df.drop('Response', axis=1)
    y_train = train_df['Response']
    
    # Split the data for validation
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.7, random_state=42)
    
    # Train using logistic regression
    model = train_logistic_regression_model(X_train, y_train)
    evaluate_model(model, X_test, y_test)
    
    # Train using SVM
    model = train_svm_model(X_train, y_train)
    evaluate_model(model, X_test, y_test)
    
def train_logistic_regression_model(X_train, y_train):
    print("\n")
    print("Traning LogisticRegression model...")
    model = LogisticRegression(solver='liblinear')
    model.fit(X_train, y_train)
    return model

def train_svm_model(X_train, y_train):
    print("Traning SVC model...")
    model = SVC()
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_test, y_test):
    print(f"Evaluating {type(model).__class__.__name__} model...")
    
    predictions = model.predict(X_test)
    accuracy = np.average(y_test == predictions)
    print("Accuracy:", accuracy)
    
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test, y_scores[:,1])
    print("AUC:", auc)
    print("*" * 30)

def read_data(data_path):
    # Read data
    data = pd.read_csv(data_path, index_col=0)
    return data

if __name__ == "__main__":
    
    print("\n")
    print("*" * 60)

    args = parse_args()
    main(args)
    
    print("*" * 60)
    print("\n")

Overwriting src/train.py
